In [1]:
from aocd import get_data, submit
# from aocd.get import current_day

from IPython.display import display
from IPython.core.display import Markdown

import math
import numpy as np
import re
import sys
import time

In [2]:
DAY = 17
YEAR = 2023

In [3]:
url = f"https://adventofcode.com/{YEAR}/day/{DAY}"
display(Markdown(f"#### See [{YEAR} Day {DAY}]({url})."))

#### See [2023 Day 17](https://adventofcode.com/2023/day/17).

In [4]:
data = get_data(year=YEAR, day=DAY)

In [5]:
data.splitlines()[:5]

['233244231122553122525413454522354135462222326535344662264544223642652255336664663346353252645466356456462435622214553413332454432211214411143',
 '223141234553125225554452413445222253233246635553325226333542552424263422323666243555565625352563233242463262245343212134422111434232123132432',
 '222211241414332422353143154123232224435622425635432542244626326352546424262544336555243322432452265334323634435625353134233211423335325223333',
 '322232223424242153454434322156223244543255665445556362542353232362553242363465623353356322354425466433346323664633124432225444231511234544333',
 '332334545315334114453243332464466364433664255323465255433565444463354334373422362334624463322442462434232264664443442124112355243442414251234']

# Part A

In [39]:
data = """2413432311323
3215453535623
3255245654254
3446585845452
4546657867536
1438598798454
4457876987766
3637877979653
4654967986887
4564679986453
1224686865563
2546548887735
4322674655533
"""

In [40]:
data = np.array([[int(x) for x in l] for l in data.splitlines()])
data = data[:5,:5]
data

array([[2, 4, 1, 3, 4],
       [3, 2, 1, 5, 4],
       [3, 2, 5, 5, 2],
       [3, 4, 4, 6, 5],
       [4, 5, 4, 6, 6]])

s = (loss: int, path: List[tuple])

In [41]:
from heapq import heappush, heappop
from functools import cache
from itertools import product

# A*, right but slow

In [42]:
@cache
def get_neighbors(loc, dir, mom):
    banned = [(-dir[0], -dir[1])]
    if mom >= 3:
        banned += [dir]

    neighbors = [(loc[0]+d[0], loc[1] + d[1]) for d in dirs if d not in banned]
    return [n for n in neighbors if n in bounds]

@cache
def get_loc_dir_mom(*p):
    moves = []
    for c,n in zip(p,p[1:]):
        moves += [(n[0]-c[0], n[1]-c[1])]
    mom = 0
    for i in range(len(moves)):
        if moves[-1] == moves[-1-i]:
            mom += 1
    return p[-1], moves[-1] if moves else (0,0), mom

@cache
def manhattan(a,b):
    return abs(a[0]-b[0]) + abs(a[1]-b[1])

In [43]:
dirs = [(-1,0),(1, 0),(0, -1),(0, 1)]
bounds = list(product(range(data.shape[0]), range(data.shape[1])))

start = (0,0)
end = tuple([data.shape[0]-1, data.shape[1]-1])

Q = []
heappush(Q,(manhattan(start,end), [start]))

i = 0
while Q:
    loss, path = heappop(Q)
    loc = path[-1]
    if i % 100 == 0:
        print(loss, path, loc)
        printpath = np.zeros(data.shape)
        for x in path:
            printpath[x] = 1
        print(printpath)
    i += 1
    if loc == end:
        print('end')
        print(loss, path, loc)
        printpath = np.zeros(data.shape)
        for x in path:
            printpath[x] = 1
        print(printpath)
        break

    neighbors = get_neighbors(*(get_loc_dir_mom(*path[-4:])))

    for n in neighbors:
        if n not in path:
            heappush(Q, (loss - manhattan(loc,end) +  data[n] + manhattan(n,end), path + [n]))

8 [(0, 0)] (0, 0)
[[1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
20 [(0, 0), (1, 0), (2, 0), (2, 1), (1, 1), (1, 2), (0, 2), (0, 3)] (0, 3)
[[1. 0. 1. 1. 0.]
 [1. 1. 1. 0. 0.]
 [1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
23 [(0, 0), (1, 0), (2, 0), (2, 1), (3, 1), (3, 0), (4, 0)] (4, 0)
[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0.]
 [1. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]]
25 [(0, 0), (1, 0), (2, 0), (2, 1), (1, 1), (1, 2), (1, 3), (2, 3), (2, 4)] (2, 4)
[[1. 0. 0. 0. 0.]
 [1. 1. 1. 1. 0.]
 [1. 1. 0. 1. 1.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
27 [(0, 0), (0, 1), (1, 1), (2, 1), (2, 2), (2, 3), (1, 3)] (1, 3)
[[1. 1. 0. 0. 0.]
 [0. 1. 0. 1. 0.]
 [0. 1. 1. 1. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
end
28 [(0, 0), (0, 1), (0, 2), (1, 2), (1, 3), (1, 4), (2, 4), (3, 4), (4, 4)] (4, 4)
[[1. 1. 1. 0. 0.]
 [0. 0. 1. 1. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]


In [44]:
loss

28

In [45]:
path

[(0, 0), (0, 1), (0, 2), (1, 2), (1, 3), (1, 4), (2, 4), (3, 4), (4, 4)]

# Dijkstra, wrong but fast

In [47]:
dirs = [(-1,0),(1, 0),(0, -1),(0, 1)]
bounds = list(product(range(data.shape[0]), range(data.shape[1])))

start = (0,0)
end = tuple([data.shape[0]-1, data.shape[1]-1])

Q = bounds.copy()
minloss = np.ones(data.shape) * 1000
prev = {}
minloss[start] = 0

while Q:
    Q.sort(key=lambda x: minloss[x])
    s = Q.pop(0)
    if s == end:
        print('end')
        break

    neighbors = [(s[0]+d[0], s[1] + d[1]) for d in dirs]
    hist = []
    c = s
    for i in range(3):
        l = prev.get(c, (-5,-5))
        m = (c[0]-l[0], c[1]-l[1])
        hist.append(m)
        c = l
    if len(set(hist)) == 1:
        banned = (s[0]+m[0], s[1]+m[1])
    else:
        banned = []
    neighbors = [n for n in neighbors if n in Q and n != banned]
    for n in neighbors:
        v = minloss[s] + data[n]
        if v < minloss[n]:
            minloss[n] = v
            prev[n] = s

path = [end]
s = end
while s != start:
    last = prev[s]
    path.append(last)
    s = last
path.reverse()

end


In [49]:
printpath = np.zeros(data.shape)
for x in path:
    printpath[x] = 1
print(printpath)

[[1. 0. 0. 0. 0.]
 [1. 1. 1. 1. 0.]
 [0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]


In [50]:
minloss[path[-1]]

29.0

# DFS, right but slow

In [54]:
def dfs(pastlocs, pastmoves, s):
    if s == end:
        return 0
    if s in pastlocs:
        return 1e7
        
    banned = []
    if len(pastmoves) >= 3 and len(set(pastmoves[-3:])) == 1:
        banned += [pastmoves[-1]]

    ans = 1e7

    for d in dirs:
        if d not in banned:
            n = (s[0]+d[0], s[1] + d[1])
            if n in bounds:
                ans = min(ans, data[n] + dfs(pastlocs + [s], pastmoves + [d], n))
    return ans

In [55]:
sys.setrecursionlimit(1500)

dfs([], [(0,0)], start)

28

In [ ]:
submit(total, year=YEAR, day=DAY)

# Part B

In [ ]:
submit(total, year=YEAR, day=DAY)